In [ ]:
import os
import json

def coco_to_yolo_segmentation(coco_json_path, output_dir):
    # Load COCO JSON
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Prepare a dictionary for image dimensions
    image_dimensions = {img['id']: (img['width'], img['height']) for img in coco_data['images']}

    # Iterate through annotations
    for annotation in coco_data['annotations']:
        image_id = annotation['image_id']
        category_id = annotation['category_id']
        segmentation = annotation['segmentation']
        bbox = annotation['bbox']

        # Get image dimensions
        width, height = image_dimensions[image_id]

        # Normalize bbox (x_center, y_center, width, height)
        x_min, y_min, box_width, box_height = bbox
        x_center = (x_min + box_width / 2) / width
        y_center = (y_min + box_height / 2) / height
        box_width /= width
        box_height /= height

        # Normalize segmentation points
        normalized_segmentation = []
        for seg in segmentation:
            normalized_segmentation.extend([seg[i] / width if i % 2 == 0 else seg[i] / height for i in range(len(seg))])

        # Combine class, bbox, and segmentation points
        yolo_format = [category_id, x_center, y_center, box_width, box_height] + normalized_segmentation

        # Save to YOLO format file
        output_file = os.path.join(output_dir, f"{image_id:06d}.txt")
        with open(output_file, 'a') as f:
            f.write(" ".join(map(str, yolo_format)) + "\n")

    print(f"Conversion completed. YOLO files saved in {output_dir}")

# Example usage
coco_json_path = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\Data\PCI_Raw\labels\__1JSTPR_S04_2024-08-21-01h24m55s544_crop_0.json' # Path to your COCO JSON file
output_dir = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\Data\PCI_Final\segmentation'         # Directory to save YOLO segmentation labels
coco_to_yolo_segmentation(coco_json_path, output_dir)


In [ ]:
from ultralytics.data.converter import convert_coco

convert_coco(labels_dir=r'C:\Users\fbpza\Desktop\Road-Lines-Detection\Data\PCI_Raw\labels', use_segments=True)

In [ ]:
import os
import json
import shutil

# load json and save directory for labels train/val/test
coco_file = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\Data\PCI_Raw\labels\__1JSTPR_S04_2024-08-21-01h24m55s544_crop_0.json'
save_folder = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\Data\PCI_Final\segmentation'


#source of all the images and destination folder for train/test/val
source_path = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\Data\PCI_Raw\labels'
destination_path = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\Data\PCI_Final\segmentation'


# Use os.listdir() to get a list of filenames in the folder
file_names = os.listdir(source_path)

with open(coco_file) as f:
    coco = json.load(f)
print(coco.keys())
images = coco['images']
annotations = coco['annotations'] 
categories = {cat['id']: cat['name'] for cat in coco['categories']}

#os.makedirs(save_folder, exist_ok=True)

for ann in annotations:
    image = next(img for img in images if (img['id'] == ann['image_id']))
    if (image["file_name"] not in file_names):
        continue
    #print(f"image in annotations =   {type(image['id'])}")
    width, height = image['width'], image['height']
    x_center = (ann['bbox'][0] + ann['bbox'][2] / 2) / width
    y_center = (ann['bbox'][1] + ann['bbox'][3] / 2) / height
    bbox_width = ann['bbox'][2] / width
    bbox_height = ann['bbox'][3] / height
    category_id = ann['category_id']
    image_id = ann['image_id']

    filename = image['file_name']
    label_filename = filename.split('.jpg')[0]
    label_path = os.path.join(save_folder, f'{label_filename}.txt')
    with open(label_path, 'a') as f:

        segmentation_points_list = []
        for segmentation in ann['segmentation']:
            # Check if any element in segmentation is a string
            if any(isinstance(point, str) for point in segmentation):
                continue  # Skip this segmentation if it contains strings

            segmentation_points = [str(float(point) / width) for point in segmentation]
            segmentation_points_list.append(' '.join(segmentation_points))
            segmentation_points_string = ' '.join(segmentation_points_list)
            line = '{} {}\n'.format(categories, segmentation_points_string)
            f.write(line)
            segmentation_points_list.clear()
    image_source = source_path + f'/{image["file_name"]}'
    shutil.copy(image_source, destination_path)

In [8]:
import os
import json
import shutil

# Directory containing JSON files and save directory for labels train/val/test
json_folder = r'C:\Users\fbpza\Desktop\pci_SEG dATA\mUAVIA\New PCI segmentations'
save_folder = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\Data\PCI_Final\segmentation'

# Source of all the images and destination folder for train/test/val
source_path = r'C:\Users\fbpza\Desktop\pci_SEG dATA\mUAVIA\New PCI segmentations'
destination_path = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\Data\PCI_Final\segmentation'

# Use os.listdir() to get a list of filenames in the folder
file_names = os.listdir(source_path)

# Create save folder if it doesn't exist
os.makedirs(save_folder, exist_ok=True)

# Iterate over all JSON files in the json_folder
for json_file in os.listdir(json_folder):
    if json_file.endswith('.json'):
        json_path = os.path.join(json_folder, json_file)
        with open(json_path) as f:
            data = json.load(f)

        shapes = data['shapes']
        image_path = data['imagePath']
        image_file_name = os.path.basename(image_path)

        if image_file_name not in file_names:
            continue

        # Assuming you have width and height information in the JSON data
        width = data['imageWidth']
        height = data['imageHeight']

        for shape in shapes:
            label = shape['label'].split('_')[0]
            points = shape['points']

            segmentation_points_list = []
            for point in points:
                x, y = point
                # Normalize the coordinates
                normalized_x = x / width
                normalized_y = y / height
                segmentation_points_list.append(f"{normalized_x} {normalized_y}")

            segmentation_points_string = ' '.join(segmentation_points_list)
            label_filename = image_file_name.split('.jpg')[0]
            label_path = os.path.join(save_folder, f'{label_filename}.txt')
            with open(label_path, 'a') as f:
                line = f'{label} {segmentation_points_string}\n'
                f.write(line)

        image_source = os.path.join(source_path, image_file_name)
        shutil.copy(image_source, destination_path)

In [5]:
import os

# Define the directory containing the annotation files
annotation_dir = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\Data\PCI_Final\segmentation'
output_dir = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\Data\PCI_Final\segmentation'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# List all annotation files in the directory
annotation_files = [f for f in os.listdir(annotation_dir) if f.endswith('.txt')]

# Dictionary for class ID replacement
class_id_dict = {
    'ALLIGATOR': '9',
    'BLOCK': '1',
    'LONGITUDINAL': '2',
    'PATCHING': '6',
    'POTHOLE': '5',
    'RAVELING': '10',
    'RAVELING.': '4',
    'TRANSVERSE': '3',
    'WEATHERING': '7'
}

# Function to convert a single annotation line
def convert_annotation_line(line):
    parts = line.strip().split()
    class_key = parts[0].split('_')[1]  # Extract class key from the first part
    class_id = class_id_dict.get(class_key, parts[0].split('_')[0])  # Replace class ID using the dictionary
    coordinates = parts[1:]  # Remaining parts are coordinates
    # Normalize coordinates (assuming image size is known, e.g., 1024x1024)
    image_width, image_height = 1024, 1024
    normalized_coords = [str(float(coord) / image_width if i % 2 == 0 else float(coord) / image_height) for i, coord in enumerate(coordinates)]
    return f"{class_id} " + " ".join(normalized_coords)

# Iterate over each annotation file
for annotation_file in annotation_files:
    input_path = os.path.join(annotation_dir, annotation_file)
    output_path = os.path.join(output_dir, annotation_file)
    
    with open(input_path, 'r') as infile, open(output_path, 'w') as outfile:
        for line in infile:
            converted_line = convert_annotation_line(line)
            outfile.write(converted_line + '\n')